In [6]:
import pandas as pd
import numpy as np
from .utils import *

pd.set_option('display.max_columns', None)

ImportError: attempted relative import with no known parent package

In [2]:
DATA_PATH = '../data/'
PASSES_DF_PATH = DATA_PATH + 'passes_df.pkl'
WYSCOUT_TO_SKILLCORNER = DATA_PATH + 'wyscout2skillcorner.csv'

In [3]:
passes_df = pd.read_pickle(PASSES_DF_PATH)
wyscout2skillcorner = pd.read_csv(WYSCOUT_TO_SKILLCORNER).drop(columns='id')

In [4]:
passes_df['responsibility'] = passes_df.apply(responsibility, axis=1, pass_length_factor = 1)

REORDER THESE 2

In [6]:
# First, merge id_map with joined_df on player.id (WyScout player IDs) to get the team_name_sk for player_skillcorner_id
passes_df = passes_df.merge(wyscout2skillcorner[['player_id_wy', 'player_id_sk', 'team_name_sk']],
                            left_on='player.id', 
                            right_on='player_id_wy', 
                            how='left')

# Rename the columns for clarity
passes_df.rename(columns={'player_id_sk': 'player_skillcorner_id', 'team_name_sk': 'team_name_skillcorner'}, inplace=True)

# Now merge id_map with object_id (which is SkillCorner ID) to get the team_name_sk for object_id
passes_df = passes_df.merge(wyscout2skillcorner[['player_id_sk', 'team_name_sk']],
                            left_on='tracking.object_id', 
                            right_on='player_id_sk', 
                            how='left')

# Rename the columns for clarity
passes_df.rename(columns={'team_name_sk': 'team_name_object'}, inplace=True) 

# Finally, create a new column to check if both players are on the same team
passes_df['same_team'] = passes_df['team_name_skillcorner'] == passes_df['team_name_object']

# You can drop any unnecessary columns if needed
passes_df.drop(columns=['player_id_wy', 'player_id_sk'], inplace=True)

In [7]:
passes_df['responsibility'] = np.where(passes_df['same_team'], 0, passes_df['responsibility'])